In [68]:
import os,sys,inspect
from ipywidgets import interact, interactive, fixed, interact_manual, Button
from IPython.display import display, Markdown, Latex, Javascript, clear_output, HTML
import emoji
import time
from datetime import date
import ipywidgets as widgets
import ipython_blocking
import numpy as np
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )

display(Markdown('### 🦢 Проверка устойчивости к механическому воздействию'))

### 🦢 Проверка устойчивости к механическому воздействию

In [69]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

Выбранная конфигурация
filter_if:10 k
point_value:200
athenuator_value:10
cap_value:6554


<IPython.core.display.Javascript object>

In [78]:
button = box.select_device_continue_button

In [79]:
get_ipython().run_line_magic('blockrun', 'button')

In [80]:
button1 = box.select_config_continue_button

In [81]:
get_ipython().run_line_magic('blockrun', 'button1')

In [82]:
inst = box.selected_device

In [83]:
result_config =  box.config

In [84]:
def HZ_convert(pref, hertz):
    try:
        hertz = int(hertz)
    except ValueError:
        return -1
    if pref == "k":
        return hertz * 1000
    if pref == "M":
        return hertz * 1000000
    if pref == "G":
        return hertz * 1000000000
    return -1

def one_scan(inst):
    inst.write('TRIG:SOUR BUS')  # устанавливаем источник триггера на шину
    inst.write(f"INIT:CONT OFF")  # включаем повторный режим запуска триггера
    # переходим в состояние ожидания триггера
    inst.write('INIT')
    inst.write('TRIG:SING')  # выполняем россчерк
    inst.query('*OPC?') 

In [85]:
value_dict = {}

In [86]:
out = widgets.HTML()


In [87]:
if_split = result_config['DEVICE']['filter_IF'].split(" ")
if_value = f'{HZ_convert(if_split[1], int(if_split[0]))}'
at_value = result_config['DEVICE']['athenuator_value']
poin_value = result_config['DEVICE']['point_value']
cap_value = result_config['DEVICE']['cap_value']   

In [88]:
def param(if_value,poin_value,at_value,cap_value):
    inst.write(f'SYST:PRES')
    inst.write(f'SENS:SWE:TYPE LIN')
    
    #Включаем аттюнатор 
    inst.write(f'SERV:RFCTL:POW:STAT 1')
    inst.write(f'SERV:RFCTL:POW:ATT {at_value}')

    #Фильтр ПЧ
    inst.write(f'SENS:BWID {if_value}')
    time.sleep(1) 

    #Код ЦАП
    inst.write(f'SERV:RFCTL:POW:DAC {cap_value}')
    
    #количество точек.
    inst.write(f'SENS:SWE:POIN {poin_value}')


In [89]:
def setport(port):
    inst.write(f'CALC1:PAR:COUN 1')
    inst.write(f'CALC1:PAR1:DEF S{port}{port}')

In [90]:
def normalize(inst):
    inst.write(f'CALC:PAR1:SEL')
    inst.write(f'CALC:MATH:MEM')
    inst.write(f'CALC:MATH:FUNC DIV')
    

In [91]:
def gridbutton(count, value_dict):
    row = 0
    port_count = 0
    
    if count%6 == 0:
        row_1 = count//6
    else:
        row_1 = count//6+1
    c = 6
    grid = widgets.GridspecLayout(row_1 * 3, 6)
    for i in range(row_1):
        if i == row_1 - 1:
            c = count - c * i
        for j in range(c):
            port_count += 1
            g1 = widgets.Button(description=f'До воздействия')
            g1.on_click(functools.partial(before_on_button_clicked, rs_=f'{port_count}'))

            g2 =  widgets.Button(description=f'После воздействия')
            g2.on_click(functools.partial(after_on_button_clicked, value_dict=value_dict, rs_=f'{port_count}'))
            grid[row + 0, j] = widgets.Label(value=f'port {port_count}')
            grid[row + 1, j] = g1
            grid[row + 2, j] = g2
        row += 3
    display(grid)


In [92]:
param(if_value,poin_value,at_value,cap_value)


date_inf = f'{date.today()}'
time_inf = inst.query(f'SYST:TIME?')

In [93]:
def marksearch(value_dict):
        
    trace = inst.query(f'CALC1:PAR1:DEF?')
    inst.write(f'CALC1:TRAC1:MARK1:STAT ON')
    inst.write(f'CALC1:TRAC1:MARK2:STAT ON')
    inst.write(f'CALC1:MARK1:FUNC:TYPE MAX')
    inst.write(f'CALC1:MARK2:FUNC:TYPE MIN')

    inst.write(f'CALC1:MARK1:FUNC:EXEC')
    inst.write(f'CALC1:MARK2:FUNC:EXEC')

    
    max_value = format(abs(float(inst.query(f'CALC1:MARK1:Y?').split(',')[0])),'.9f')
    min_value = format(abs(float(inst.query(f'CALC1:MARK2:Y?').split(',')[0])),'.9f')
    if max_value > min_value:
        value_dict.update({f'{trace}': max_value})
    else:
        value_dict.update({f'{trace}': min_value})
    return value_dict

In [94]:
def tablehtml(value_dict): 
    htmlstr = "<table border='1' ><tr><td>Порт</td><td>Отклонение</td><td>Максимум, дБ</td></tr><tr>"
    count = 0
    MAX = -1
    for trace in value_dict.keys():
        if float(value_dict[f'{trace}']) > float(MAX):
            MAX = value_dict[f'{trace}']
    for trace in value_dict.keys():
        trace_value = value_dict[f'{trace}']
        cut = (len(trace)-1)//2 + 1
        port_data = trace[cut:]
        if trace_value == MAX:
            htmlstr += f'<td>{port_data}</td><td class = bg-warning>{trace_value}</td>'
        else:
            htmlstr += f'<td>{port_data}</td><td>{trace_value}</td>'
        if count == 0:
            htmlstr += f'<td rowspan={len(value_dict.keys())+1}>{MAX}</td>'
            count += 1
        htmlstr += "</tr><tr>"
    htmlstr += '''</tr><style type="text/css">
td{
    padding: 2px;
    border-collapse: collapse;
    background-color:palegreen;
}
tr{
   padding: 2px;
   color: blue;
   border-collapse: collapse;
   font-family:verdana;
}
tr{
   font-size:20px;
   color:black;
   border-collapse: collapse;
   text-align:center;
}
</style>'''
    return htmlstr



In [95]:
def teblehader():
    
    idn_inf = inst.query(f'*IDN?').split(', ')
    htmlstr = "<table border='1' ><tr><td> Дата запуска </td><td> Время запуска </td><td>  device </td><td> model </td><td> serial </td><td>  version </td></tr><tr>"
    htmlstr += f'<td>{date_inf}</td><td>{time_inf}</td><td>{idn_inf[0]}</td><td>{idn_inf[1]}</td><td>{idn_inf[2]}</td><td>{idn_inf[3]}</td>'
    return htmlstr

In [96]:
def before_on_button_clicked(b, rs_="some_default_string"):
    setport(rs_)
    one_scan(inst)
    normalize(inst)
    one_scan(inst)
    b.button_style = 'success'

In [97]:
def after_on_button_clicked(b, value_dict, rs_="some_default_string"):
    setport(rs_)
    one_scan(inst)
    marksearch(value_dict)
    out.value = teblehader() + tablehtml(value_dict)
    b.button_style = 'success'
    global save_button
    save_button.file_data = teblehader() + tablehtml(value_dict)


In [98]:

gridbutton(int(inst.query(f'SERV:PORT:COUN?')), value_dict)
display(out)
save_button = SaveFilesButton(out.value)
display(save_button)


GridspecLayout(children=(Label(value='port 1', layout=Layout(grid_area='widget001')), Button(description='До в…

HTML(value='')

SaveFilesButton(description='SaveFiles', icon='square-o', style=ButtonStyle(button_color='orange'))